<a href="https://colab.research.google.com/github/JozefSL/2022/blob/main/dbf2sqlite_3d_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dbfread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.1 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 


In [10]:
import pandas as pd
import plotly.express as px
from dbfread import DBF
import glob
import sqlite3
import sys
import csv
import dbfread
import dataset

In [3]:
!wget https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip

--2022-07-22 11:03:53--  https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip
Resolving gis.dmr.nd.gov (gis.dmr.nd.gov)... 165.234.156.197
Connecting to gis.dmr.nd.gov (gis.dmr.nd.gov)|165.234.156.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293655700 (280M) [application/x-zip-compressed]
Saving to: ‘OGD_Horizontals.zip’

OGD_Horizontals.zip 100%[===================>] 280.05M  77.4MB/s    in 3.8s    

2022-07-22 11:03:57 (73.3 MB/s) - ‘OGD_Horizontals.zip’ saved [293655700/293655700]



In [4]:
!unzip OGD_Horizontals.zip

Archive:  OGD_Horizontals.zip
 extracting: OGD_Horizontals.cpg     
  inflating: OGD_Horizontals.dbf     
  inflating: OGD_Horizontals.prj     
  inflating: OGD_Horizontals.sbn     
  inflating: OGD_Horizontals.sbx     
  inflating: OGD_Horizontals.shp     
  inflating: OGD_Horizontals.shp.xml  
  inflating: OGD_Horizontals.shx     


In [5]:
filename= 'OGD_Horizontals.dbf'



In [12]:
db = dataset.connect('sqlite:///:memory:')
table = db['wells']
for record in DBF(filename, lowernames=True):
  table.insert(record)


In [14]:
print(db.tables)

['wells']


In [15]:
print(db['wells'].columns)

['id', 'wl_permit', 'api_wellno', 'api_format', 'bh_long', 'bh_lat', 'well_sub', 'measdpth', 'bh_inc', 'bh_azmuth', 'tvd', 'bh_coordns', 'bh_coord_1', 'bh_coordew', 'bh_coord_2', 'apitype']


In [16]:
print(table.find_one(api_wellno = '33007010420000'))

OrderedDict([('id', 5692), ('wl_permit', '11335'), ('api_wellno', '33007010420000'), ('api_format', '33-007-01042-00-00'), ('bh_long', -103.51863817), ('bh_lat', 46.86768147), ('well_sub', 'LAT1'), ('measdpth', 10720.0), ('bh_inc', 89.80000305), ('bh_azmuth', 238.8999939), ('tvd', 9013.20996094), ('bh_coordns', 874.48950195), ('bh_coord_1', 'S'), ('bh_coordew', 1588.00549316), ('bh_coord_2', 'W'), ('apitype', '33007010420000_LAT1')])


In [36]:
plWell = table.find(api_wellno = '33023003280000')


In [56]:
plWell = table.find(api_wellno = '33023003280000')
fig = px.scatter_3d(plWell, x='bh_long', y='bh_lat', z='tvd', color='well_sub')
fig.update_traces(marker_size = 3)
fig.update_scenes(yaxis_autorange="reversed")
fig.update_scenes(xaxis_autorange="reversed")
fig.update_scenes(zaxis_range=[16000, 0])
fig.update_layout({
'plot_bgcolor': 'rgba(100, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
table = DBF(filename)
writer = csv.writer(sys.stdout)
writer.writerow(table.field_names)
for record in table:
  writer.writerow(list(record.values()))

In [6]:
%pd

UsageError: Line magic function `%pd` not found.


In [13]:
%ls

OGD_Horizontals.cpg  OGD_Horizontals.sbn  OGD_Horizontals.shp.xml  sample_data/
OGD_Horizontals.dbf  OGD_Horizontals.sbx  OGD_Horizontals.shx
OGD_Horizontals.prj  OGD_Horizontals.shp  OGD_Horizontals.zip


In [22]:
    conn = sqlite3.connect('ND_hzdata')
    cursor = conn.cursor()


In [ ]:
db = conn.connect('sqlite:///:memory:')


In [30]:
db = sqlite3.connect('ND_hzdata')
cursor = db.cursor()

In [36]:
dbf2sqlite(filename)

NameError: ignored

In [ ]:
for record in DBF(filename, lowernames=True):
  table.insert(record)

In [26]:

for record in DBF('files/people.dbf', lowernames=True):
table.insert(record)

    db = dataset.connect('sqlite:///:memory:')
    table = db['people']
 table = DBF(filename)

IndentationError: ignored

In [21]:
for record in DBF(filename, lowernames=True):
  print(record)

Streaming output truncated to the last 5000 lines.
OrderedDict([('wl_permit', '5343'), ('api_wellno', '33007001570000'), ('api_format', '33-007-00157-00-00'), ('bh_long', -103.30147225), ('bh_lat', 46.87150689), ('well_sub', 'LAT2'), ('measdpth', 9846.0), ('bh_inc', 92.0), ('bh_azmuth', 231.1000061), ('tvd', 9412.11914063), ('bh_coordns', 347.01031494), ('bh_coord_1', 'S'), ('bh_coordew', 362.50436401), ('bh_coord_2', 'W'), ('apitype', '33007001570000_LAT2')])
OrderedDict([('wl_permit', '5343'), ('api_wellno', '33007001570000'), ('api_format', '33-007-00157-00-00'), ('bh_long', -103.30236916), ('bh_lat', 46.87092832), ('well_sub', 'LAT2'), ('measdpth', 10154.0), ('bh_inc', 90.0), ('bh_azmuth', 223.3999939), ('tvd', 9412.82421875), ('bh_coordns', 557.88720703), ('bh_coord_1', 'S'), ('bh_coordew', 586.53320313), ('bh_coord_2', 'W'), ('apitype', '33007001570000_LAT2')])
OrderedDict([('wl_permit', '5343'), ('api_wellno', '33007001570000'), ('api_format', '33-007-00157-00-00'), ('bh_long', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



OrderedDict([('wl_permit', '12620'), ('api_wellno', '33007011710000'), ('api_format', '33-007-01171-00-00'), ('bh_long', -103.60761167), ('bh_lat', 47.10528746), ('well_sub', 'LAT1'), ('measdpth', 10885.0), ('bh_inc', 90.30000305), ('bh_azmuth', 181.80000305), ('tvd', 10385.3164062), ('bh_coordns', 721.39416504), ('bh_coord_1', 'S'), ('bh_coordew', 67.53468323), ('bh_coord_2', 'W'), ('apitype', '33007011710000_LAT1')])
OrderedDict([('wl_permit', '12620'), ('api_wellno', '33007011710000'), ('api_format', '33-007-01171-00-00'), ('bh_long', -103.6076195), ('bh_lat', 47.1052026), ('well_sub', 'LAT1'), ('measdpth', 10916.0), ('bh_inc', 91.59999847), ('bh_azmuth', 185.3999939), ('tvd', 10384.8027344), ('bh_coordns', 752.32299805), ('bh_coord_1', 'S'), ('bh_coordew', 69.4803009), ('bh_coord_2', 'W'), ('apitype', '33007011710000_LAT1')])
OrderedDict([('wl_permit', '12620'), ('api_wellno', '33007011710000'), ('api_format', '33-007-01171-00-00'), ('bh_long', -103.60763047), ('bh_lat', 47.1051179

KeyboardInterrupt: ignored

In [ ]:
table = DBF(filename)
writer = csv.writer(sys.stdout)
writer.writerow(table.field_names)
for record in table:
  writer.writerow(list(record.values()))

In [ ]:
for record in DBF('files/people.dbf', lowernames=True):
    table.insert(record)


In [ ]:
df = pd.DataFrame(iter(DBF(filename, load=True)))

In [ ]:
table = DBF(filename, load=True)
frame = pd.DataFrame(iter(table))

In [ ]:
fig = px.scatter_3d(df, x='bh_long', y='bh_lat', z='tvd',
              color='api_wellno')
fig.show()

In [4]:
table

NameError: ignored

In [2]:
dbf = DBF('OGD_Horizontals.dbf')

In [ ]:
wellND = pd.DataFrame(iter(dbf))

In [4]:
for filename in glob.glob('OGD_Horizontals.dbf'):
  print('reading dbf file ' + filename)

        # load the dbf file
  table = DBF(filename, load=False)

        # iterate through dbf file and store each record in a list
  listOfRecords = ['api_wellno','bh_long','bh_lat','tvd']
  for record in table:
      listOfRecords.append(record)

        # create a pandas dataframe from the list of records
      df = pd.DataFrame(listOfRecords)

return df

reading dbf file OGD_Horizontals.dbf


KeyboardInterrupt: ignored

In [8]:
filename= 'OGD_Horizontals.dbf'

table = DBF(filename, load=False)

        # iterate through dbf file and store each record in a list
listOfRecords = ['api_wellno','bh_long','bh_lat','tvd','well_sub']
for record in table:
  listOfRecords.append(record)

        # create a pandas dataframe from the list of records
  df = pd.DataFrame(listOfRecords)
return df

KeyboardInterrupt: ignored

In [ ]:
df.head()